Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

## Blatt 07

- **Jede(r)** Studierende muss die Aufgaben einreichen.
Wenn Sie die Aufgaben nicht selbst einreichen, erhalten Sie 0 Punkte für die Aufgabe.
- Sie werden ermutigt, in Gruppen von bis zu vier Personen an den Lösungen zu arbeiten.
    - Bei Einreichungen solcher Einreichungsgruppen speichern die Gruppen die imt-Logins aller Mitglieder in der Variable `AUTHORS`, z.B: `AUTHORS="first-imtlogin second-imtlogin"`.
    - Jedes Mitglied der Gruppen **muss das Assignment einreichen**.
    - Verwenden Sie dazu die vorbereitete Zelle:

**Anmerkung:** Löschen Sie *nicht* die Befehlshistorie (z.B. durch Ausführen von `history -c`) in *Ihrem* Code!

In [ ]:
AUTHORS=""

In [ ]:
[[ -d $SHEET_DIR ]] && cd $SHEET_DIR  # Ensure this cell can be executed repeatedly
rm -rf $SHEET_DIR/workdir-*  # Delete old work directories
SHEET_DIR=$(pwd)
WORK_DIR=$(mktemp -d -p $SHEET_DIR workdir-XXXXX)  # Create work directory
cd $WORK_DIR  # Change into work directory

### Aufgabe 1

Mit dem folgenden `grep`-Ausdruck kann die Modellnummer einiger gebräuchlicher Intel-CPUs (z.B. *E5-2695*) aus der `model name`-Zeile von `/proc/cpuinfo` extrahiert werden:
```Bash
grep -o "[a-zA-Z][0-9]-[0-9]\+"
```

Schreiben Sie eine Funktion `extract_model`, die diesen Befehl auf ihren `stdin`-Eingabestrom anwendet, sodass die Modellnummer im `stdout`-Ausgabestrom ausgegeben wird.

In [ ]:
# ### Aufgabe 1

# Mit dem folgenden `grep`-Ausdruck kann die Modellnummer einiger gebräuchlicher Intel-CPUs (z.B. *E5-2695*) aus der `model name`-Zeile von `/proc/cpuinfo` extrahiert werden:
# ```Bash
# grep -o "[a-zA-Z][0-9]-[0-9]\+"
# ```

# Schreiben Sie eine Funktion `extract_model`, die diesen Befehl auf ihren `stdin`-Eingabestrom anwendet, sodass die Modellnummer im `stdout`-Ausgabestrom ausgegeben wird.

extract_model() {
    grep -o "[a-zA-Z][0-9]-[0-9]\+"
}

In [ ]:
(set -e
type -t extract_model > /dev/null || (echo "function extract_model does not exist" && false)
[[ $(echo "model name: Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz" | extract_model) == "E5-2695" ]] || (echo -e "model name: Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz\nExpected model number: E5-2695\nFunction returned: $(echo "model name: Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz" | extract_model)" && false)
)

### Aufgabe 2

Schreiben Sie eine Funktion `detect_shm`, die die Größe (in `1K-blocks`) der Shared-Memory-Partition `/dev/shm` ermittelt und diese Größe in die Variable `$SHMSIZE` schreibt.

Hinweis: Partitionsgrößen können mit dem Programm `df` angezeigt werden.

In [ ]:
# ### Aufgabe 2

# Schreiben Sie eine Funktion `detect_shm`, die die Größe (in `1K-blocks`) der Shared-Memory-Partition `/dev/shm` ermittelt und diese Größe in die Variable `$SHMSIZE` schreibt.

# Hinweis: Partitionsgrößen können mit dem Programm `df` angezeigt werden.

detect_shm() {
    SHMSIZE=$(df --output=size /dev/shm | tail -n 1)
}

In [ ]:
unset SHMSIZE
(set -e
type -t detect_shm > /dev/null || (echo "function detect_shm does not exist" && false)
detect_shm
[[ -n $SHMSIZE ]] || (echo "SHMSIZE is empty" && false)
)

### Aufgabe 3.1

Schreiben Sie eine Funktion `worker`, die 10 Sekunden lang läuft und jede Sekunde die Zeile `worker <n> running` ausgibt, wobei `<n>` ein der Funktion übergebener Parameter ist.
Nach 10 Sekunden soll `done` ausgegeben werden.

In [ ]:
# ### Aufgabe 3.1

# Schreiben Sie eine Funktion `worker`, die 10 Sekunden lang läuft und jede Sekunde die Zeile `worker <n> running` ausgibt, wobei `<n>` ein der Funktion übergebener Parameter ist.
# Nach 10 Sekunden soll `done` ausgegeben werden.

worker() {
    local n=$1
    for i in $(seq 1 10); do
        echo "worker $n running"
        sleep 1
    done
    echo "done"
}

In [ ]:
(set -e
type -t worker > /dev/null || (echo "function worker does not exist" && false) 
worker 0 | tee worker.log
)

### Aufgabe 3.2

Schreiben Sie eine Funktion `start_workers`, die 10 Worker-Prozesse (0-9) im Hintergrund startet, indem sie die `worker`-Funktion aus der vorherigen Aufgabe verwendet.
Stellen Sie sicher, dass Sie die Worker-ID (0-9) als Parameter für die Funktion angeben.
Speichern Sie die PID jedes Worker-Prozesses in einem Array namens `PIDS`.

Schreiben Sie eine Funktion `wait_workers`, die auf alle zuvor gestarteten Worker-Prozesse wartet, die im Hintergrund laufen, und dann `all done` ausgibt.
Verwenden Sie dazu das Array `PIDS`, das Sie zuvor erstellt haben.

In [ ]:
# ### Aufgabe 3.2

# Schreiben Sie eine Funktion `start_workers`, die 10 Worker-Prozesse (0-9) im Hintergrund startet, indem sie die `worker`-Funktion aus der vorherigen Aufgabe verwendet.
# Stellen Sie sicher, dass Sie die Worker-ID (0-9) als Parameter für die Funktion angeben.
# Speichern Sie die PID jedes Worker-Prozesses in einem Array namens `PIDS`.

# Schreiben Sie eine Funktion `wait_workers`, die auf alle zuvor gestarteten Worker-Prozesse wartet, die im Hintergrund laufen, und dann `all done` ausgibt.
# Verwenden Sie dazu das Array `PIDS`, das Sie zuvor erstellt haben.

PIDS=()

start_workers() {
    for i in $(seq 0 9); do
        worker $i &
        PIDS+=($!)
    done
}

wait_workers() {
    for pid in ${PIDS[@]}; do
        wait $pid
    done
    echo "all done"
}


In [ ]:
(set -e
type -t start_workers > /dev/null || (echo "function start_workers does not exist" && false)
unset PIDS
start_workers
[[ -n $PIDS ]] || (echo "PIDS is empty" && false)
type -t wait_workers > /dev/null || (echo "function wait_workers does not exist" && false)
wait_workers > output
[[ $(cat output) == "all done" ]] || (echo "output does not match" && false)
cat output
rm output
)

In [ ]:
rm -rf $WORK_DIR